In [15]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai.like import OpenAILike
from agno.utils.pprint import pprint_run_response
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.openai import OpenAIEmbedder
from agno.knowledge.docx import DocxKnowledgeBase
from pathlib import Path

import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen3-235b-a22b'
embedding_model_id = 'text-embedding-v3'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = local_settings

In [127]:
# 百炼的接口不需要chat_template_kwargs字段，直接enable_thinking即可

#agent = Agent(model=OpenAILike(**settings))


# Run agent and return the response as a variable
# 同时会返回session_id，用于后续的连续对话，run_id表示每一轮次的对话
#agent.run("你是谁")

In [17]:
vector_db = LanceDb(
    table_name="contact_file",
    uri="tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=OpenAIEmbedder(id=embedding_model_id,api_key=api_key,base_url=base_url, dimensions=1024),
)

INFO Creating table: contact_file

In [ ]:
# Create a knowledge base with the DOCX files from the data/docs directory
# 默认的chunking_strategy是FixedSizeChunking，chunk_size=5000，overlap=0
knowledge = DocxKnowledgeBase(
    path=Path(r"D:\【个人工作】\【2025】\AI解析合同\安阳钢铁集团有限责任公司综利公司烧结机头灰资源化处置项目（运营）"),
    vector_db=vector_db,
)

In [ ]:
# 通过load加载和拆分知识库
knowledge.load(recreate=False) # 设置为True会删除原有知识库并重新创建

INFO Loading knowledge base

INFO Reading:                                                                                                      
     D:\【个人工作】\【2025】\AI解析合同\安阳钢铁集团有限责任公司综利公司烧结机头灰资源化处置项目（运营）\01三级浆 
     液泵泵头采购合同.docx

INFO Skipped 3 existing/duplicate documents.

INFO Added 0 documents to knowledge base

INFO Reading:                                                                                                      
     D:\【个人工作】\【2025】\AI解析合同\安阳钢铁集团有限责任公司综利公司烧结机头灰资源化处置项目（运营）\02       
     阀门补充采购合同.docx

...

INFO Skipped 3 existing/duplicate documents.

INFO Added 0 documents to knowledge base

INFO Reading:                                                                                                      
     D:\【个人工作】\【2025】\AI解析合同\安阳钢铁集团有限责任公司综利公司烧结机头灰资源化处置项目（运营）\05       
     压滤机滤布采购合同.docx

INFO Skipped 3 existing/duplicate documents.

INFO Added 0 documents to knowledge base

In [125]:
agent = Agent(model=OpenAILike(**settings), knowledge=knowledge, 
    # Enable RAG by adding references from AgentKnowledge to the user prompt.
    add_references=True,
    # Add a tool to search the knowledge base which enables agentic RAG.
    search_knowledge=True,
    # Add a tool to read chat history.
    read_chat_history=True,
    show_tool_calls=True,
    markdown=True)

In [121]:
agent.run("烧结机头合同采购了哪些设备", markdown=True)

INFO Found 5 documents

RunResponse(content='\n\n根据提供的知识库信息，烧结机头灰资源化处置项目（运营）相关的合同采购了以下设备：\n\n### 1. **压滤机备件**\n   - **供应商**：景津装备股份有限公司\n   - **合同编号**：AGJTH2024BJ-05\n   - **合同金额**：人民币21,156.00元（含税）\n\n### 2. **阀门**\n   - **供应商1**：东亚阀门科技集团有限公司  \n     **合同编号**：AGJTH2024BJ-02  \n     **合同金额**：人民币16,520.00元（含税）\n   - **Supplier2**：双恒阀门集团有限公司  \n     **合同编号**：AGJTH2023BJ-02  \n     **合同金额**：人民币29,550.00元（含税）\n\n### 3. **循环泵备件**\n   - **Supplier**：四川省自贡工业泵有限责任公司  \n     **合同编号**：AGJTH2023BJ-04  \n     **合同金额**：人民币30,500.00元（含税）\n\n### 4. **工艺渣浆泵及泵头机封**\n   - **Supplier**：襄阳五二五泵业有限公司  \n     **合同编号**：AGJTH2024-04  \n     **合同金额**：人民币30,430.00元（含税）\n\n---\n\n### 项目背景\n所有合同均服务于 **安阳钢铁集团有限责任公司烧结机头灰资源化处置项目（运营）**，涉及设备包括压滤机、阀门、循环泵、渣浆泵及其配套部件。合同条款包含廉洁协议、安全责任划分及付款条件等。\n\n如需更详细的设备规格或合同条款，可参考对应合同文件。', content_type='str', thinking=None, reasoning_content=None, event='RunResponse', messages=[Message(role='system', content='<additional_information>\n- Use markdown to format your answers.\n</additional_information>', name=None, t

In [126]:
# Run agent and return the response as a stream
response_stream: Iterator[RunResponse] = agent.run("烧结机头合同采购了哪些设备", stream=True)
for chunk in response_stream:
    print(chunk.content,end="", flush=True)

INFO Found 5 documents



根据提供的参考资料，烧结机头相关合同采购的设备及供应商如下：

### 1. **压滤机滤布**
   - **合同编号**：AGJTH2024BJ-05  
   - **供应商**：景津装备股份有限公司  
   - **项目**：安阳钢铁烧结机头烟气脱硫脱硝系统制酸废水处理项目  
   - **合同金额**：含税总价21,156元（人民币）  

---

### 2. **阀门**
   - **合同编号**：AGJTH2024BJ-02（补充采购）、AGJTH2023BJ-02（首次采购）  
   - **供应商**：东亚阀门科技集团有限公司、双恒阀门集团有限公司  
   - **项目**：烧结机头灰资源化处置项目  
   - **合同金额**：  
     - 补充采购：16,520元（人民币）  
     - 首次采购：29,550元（人民币）  

---

### 3. **循环泵备件**
   - **合同编号**：AGJTH2023BJ-04  
   - **供应商**：四川省自贡工业泵有限责任公司  
   - **项目**：烧结机头灰资源化处置项目  
   - **合同金额**：30,500元（人民币）  

---

### 4. **工艺渣浆泵及泵头机封**
   - **合同编号**：AGJTH2024-04  
   - **供应商**：襄阳五二五泵业有限公司  
   - **项目**：烧结机头灰资源化处置项目  
   - **合同金额**：30,430元（人民币）  

---

### 总结
烧结机头合同采购的设备包括：  
1. 压滤机滤布（景津装备）  
2. 阀门（东亚阀门、双恒阀门）  
3. 循环泵备件（自贡工业泵）  
4. 工艺渣浆泵及泵头机封（襄阳五二五泵业）  

这些设备均服务于安阳钢铁烧结机头灰资源化处置项目。

In [2]:
import lancedb

In [3]:
db_path = "tmp/contact_vectors.lancedb"
# --- 1. 连接/创建 LanceDB 数据库 ---
db = lancedb.connect(db_path)
print(f"LanceDB connected at: {db.uri}")

LanceDB connected at: e:\PythonProject\LLM-Explore\src\agent\agno-usage\knowledge\tmp\contact_vectors.lancedb


In [ ]:
# --- 2. 打开表 ---
table_name = "contact_vectors" # 你之前创建的表名
try:
    table = db.open_table(table_name)
    print(f"Successfully opened table: '{table.name}'")
    print(f"Table schema: {table.schema}")
    print(f"Number of rows in table: {len(table)}")
except Exception as e:
    print(f"Error opening table '{table_name}': {e}")
    print(f"Available tables: {db.table_names()}")
    exit()

Error opening table 'contact_vectors': Table 'contact_vectors' was not found
Available tables: []


: 

In [ ]:
import json

def read_and_decode_table(db, table_name):
    table = db.open_table(table_name)
    df = table.to_pandas()
    
    if 'payload' not in df.columns:
        raise ValueError("DataFrame中缺少'payload'列")
    
    # 解析payload字符串为字典
    df['parsed_payload'] = df['payload'].apply(
        lambda x: json.loads(x) if isinstance(x, str) else x
    )
    
    # 直接提取已解码的字段值
    df['name_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    
    df['content_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('content') if isinstance(x, dict) else None
    )
    
    # 可选：删除临时解析列
    df = df.drop(columns=['parsed_payload'])
    
    return df

In [1]:
data = read_and_decode_table(db, table_name)
data

NameError: name 'read_and_decode_table' is not defined

In [ ]:
data.iloc[0]['content_cn']

'买方合同编号：AGJTH2023BJ-01 卖方合同编号： 订 货 合 同 项目名称：安阳钢铁集团有限责任公司 烧结机头灰资源化处置项目（运营） 子项名称：浆液泵泵头 买方：湖南中冶长天节能环保技术有限公司 卖方：襄阳五二五泵业有限公司 合同签订时期：2023年 8月 合同签订地点：湖南·长沙 廉洁协议 买方：湖南中冶长天节能环保技术有限公司 卖方：襄阳五二五泵业有限公司 根据党和国家有关法律法规的各项规定，为保持廉洁自律，倡导廉洁从业，有效遏制商业贿赂行为，防止在合同签订、履行中不廉洁行为的发生，经双方协商，订立本协议，相互约束。 第一条 买卖双方应当自觉遵守党和国家有关法律法规和有关廉洁自律的各项规定。 第二条 买方工作人员应当保持与卖方的正常工作和业务往来，不得接受卖方的礼金、有价证券和物品，不得在卖方报销任何应由私人支付的费用。 第三条 买方工作人员不得参加卖方的宴请和娱乐活动，不得违反廉洁从业的有关规定，不得接受任何形式的商业贿赂。 第四条 买方工作人员不得要求或接受卖方为其住房装修、婚丧嫁娶、家属和子女的工作安排及出国提供方便或任何形式的不当利益。 第五条 买方工作人员不得向卖方介绍家属或者亲友从事与本合同业务有关的经济活动。 第六条 卖方应当通过正常途径开展相关业务工作，不得向买方工作人员赠送礼金、有价证券和物品。 第七条 卖方不得为谋取私利擅自与买方工作人员私下商谈或者达成默契。 第八条 卖方不得以汇报工作、洽谈业务、签订经济合同为借口，邀请买方工作人员外出旅游、参加非正常交往的宴会、进入营业性娱乐场所。 第九条 卖方不得为买方单位和个人购置或提供通讯工具、交通工具、家具、办公用品等。 第十条 卖方如发现买方工作人员有违反上述协议者，应向买方领导或买方上级单位口头或书面举报（举报电话：中冶长天监察部 0731-82760729）。买方不得找任何借口对卖方进行报复。 第十一条 买方发现卖方有违反本协议或者采用前述不正当的手段对买方工作人员行贿或变相行贿，买方根据具体情节和造成的后果追究卖方合同标的或合同总金额1%-5%的违约金，由此给买方单位造成的损失均由卖方承担，卖方用不正当手段获取的非法所得由买方单位予以追缴，买方有权取消违反协议的卖方的投标资格或者单方面终止合同。情节严重的，列入黑名单，三年内不得进入长天市场。 第十二条 本协议作为合同附件，与

In [111]:
len(data.iloc[3]['content_cn'])

4954